In [1]:
import pandas as pd
pd.set_option("display.max_columns", 100)
import numpy as np

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [2]:
cols = ['RecordID', 'ESS_Id', 'EventTimeStamp', 'eventDescription', 'spn', 'fmi', 'active', 'activeTransitionCount',
            'EquipmentID', 'Latitude', 'Longitude', 'LocationTimeStamp']
faults = pd.read_csv("../data/J1939Faults.csv", usecols = cols, dtype = str)
diagnostics = pd.read_csv("../data/VehicleDiagnosticOnboardData.csv")
sfc = pd.read_excel("../data/Service Fault Codes_1_0_0_167.xlsx")

/Users/kd/anaconda3/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [3]:
diagnostics = pd.pivot(diagnostics, 
                       columns = ['Name'], 
                       values = 'Value', 
                       index = 'FaultId')


In [4]:
diagnostics = diagnostics.reset_index()
diagnostics['FaultId'] = diagnostics['FaultId'].astype(str)

In [5]:
# Convert columns to numeric
diagnostics[['AcceleratorPedal', 'BarometricPressure', 'CruiseControlSetSpeed', 'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'FuelTemperature', 'IntakeManifoldTemperature', 'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure']] = diagnostics[['AcceleratorPedal', 'BarometricPressure', 'CruiseControlSetSpeed', 'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'FuelTemperature', 'IntakeManifoldTemperature', 'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure']].apply(pd.to_numeric, errors = 'coerce')

In [6]:
diagnostics = pd.get_dummies(diagnostics, drop_first=True, columns = ['CruiseControlActive', 'IgnStatus', 'ParkingBrake'])
#diagnostics = diagnostics[['AcceleratorPedal', 'BarometricPressure', 'CruiseControlSetSpeed', 'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'FuelTemperature', 'IntakeManifoldTemperature', 'LampStatus','ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure', 'CruiseControlActive_True', 'IgnStatus_True', 'ParkingBrake_True']]
#diagnostics_orig = diagnostics.copy()

In [7]:
diagnostics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187335 entries, 0 to 1187334
Data columns (total 25 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   FaultId                    1187335 non-null  object 
 1   AcceleratorPedal           531880 non-null   float64
 2   BarometricPressure         585952 non-null   float64
 3   CruiseControlSetSpeed      576458 non-null   float64
 4   DistanceLtd                585798 non-null   float64
 5   EngineCoolantTemperature   586054 non-null   float64
 6   EngineLoad                 585621 non-null   float64
 7   EngineOilPressure          586220 non-null   float64
 8   EngineOilTemperature       583890 non-null   float64
 9   EngineRpm                  586909 non-null   float64
 10  EngineTimeLtd              581343 non-null   float64
 11  FuelLevel                  502777 non-null   float64
 12  FuelLtd                    585173 non-null   float64
 13  FuelRate    

In [8]:
diagnostics

,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,CruiseControlActive_True,IgnStatus_True,ParkingBrake_True
0,1,0.0,14.2100,66.48672,423178.70000,100.4,11.0,0.00,96.74375,0.00,1632.20,43.2,12300.907429,0.000000,NaN,78.8,1023,NaN,0.00000,3276.75,NaN,0.00,0,0,1
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,0,1,0
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,0,0,0
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,0,1,0
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,0,0,0
1187331,1248455,100.0,14.5000,64.62260,423937.90000,185.0,51.0,37.12,211.49370,1310.25,10722.70,96.4,58979.184416,7.647805,32.0,98.6,18431,NaN,65.01096,NaN,73.2,7.83,1,1,0
1187332,1248456,0.0,14.3550,66.48672,465925.40000,186.8,62.0,41.18,212.84380,1340.75,9326.75,100.0,65080.105870,8.995086,NaN,91.4,17407,NaN,66.57410,NaN,100.0,6.96,1,1,0
1187333,1248457,1.6,14.4275,67.72946,28606.65625,181.4,0.0,27.26,221.73120,863.25,586.75,23.6,4042.492826,0.000000,NaN,100.4,1023,NaN,11.84489,14.10,100.0,1.74,0,1,0


**There are service locations at (36.0666667, -86.4347222), (35.5883333, -86.4438888), and (36.1950, -83.174722), so you should remove any records in the vicinity of these locations, as fault codes may be tripped when working on the vehicles.**

In [9]:
station_1 = (faults['Latitude'].str.contains('36.06')) & faults['Longitude'].str.contains('86.43')
station_2 = (faults['Latitude'].str.contains('35.58')) & faults['Longitude'].str.contains('86.44')
station_3 = (faults['Latitude'].str.contains('36.19')) & faults['Longitude'].str.contains('83.17')
faults = faults.drop(faults[station_1 | station_2 | station_3].index)

In [10]:
# creating a column that has the lat long values together.
faults["lat_long"] = list(zip(faults['Latitude'], faults['Longitude']))

In [11]:
# convert dtypes as needed
# some columns have numeric-appearing values but they're not continuous variables, leaving them as strings
faults['EventTimeStamp'] = pd.to_datetime(faults['EventTimeStamp'])
faults['activeTransitionCount'] = faults['activeTransitionCount'].astype(int)

In [12]:
# time of day for faults could be interesting, split event column to date and timestamp columns
faults['event_date'] = faults['EventTimeStamp'].dt.date
faults['event_time'] = faults['EventTimeStamp'].dt.time

In [13]:
faults = faults.copy()

faults['event_year'] = faults['EventTimeStamp'].dt.year
faults['event_month'] = faults['EventTimeStamp'].dt.month
faults['event_day'] = faults['EventTimeStamp'].dt.day
faults['event_dayofweek'] = faults['EventTimeStamp'].dt.dayofweek
faults['event_dayname'] = faults['EventTimeStamp'].dt.day_name()
faults['event_hour'] = faults['EventTimeStamp'].dt.hour
faults['event_time_quadrant'] = faults['EventTimeStamp'].dt.hour // 4

In [15]:
faults['RecordID'] = faults[['RecordID']].astype(str)

Merging Dataframes

In [16]:
faults_diagnostics = pd.merge(faults, diagnostics, left_on='RecordID', right_on='FaultId')

In [17]:
diag_features=['FaultId', 'AcceleratorPedal', 'BarometricPressure', 'CruiseControlSetSpeed', 'DistanceLtd',
               'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure','EngineOilTemperature', 
               'EngineRpm', 'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'IntakeManifoldTemperature', 
               'LampStatus', 'Speed', 'TurboBoostPressure', 'CruiseControlActive_True', 'IgnStatus_True', 'ParkingBrake_True']

In [18]:
diagnostics_drop = faults_diagnostics.dropna(subset = diag_features)

In [22]:
diagnostics_drop['eventDescription'].value_counts()

Low (Severity Low) Engine Coolant Level                    104645
Abnormal Update Rate Tire Location                          69851
High Voltage (Fuel Level)                                   39678
High Voltage (Left Fuel Level Sensor)                       37927
Low (Severity Medium) Engine Coolant Level                  17032
                                                            ...  
Low Current Engine (Compression) Brake Output #1                1
Low Voltage (Engine Fuel Pump Pressurizing Assembly #1)         1
Low Current Pressure Modulation Valve ABS Axle 2 Left           1
Low (Severity High) Engine Misfire Cylinder #3                  1
Out of Calibration Program Memory                               1
Name: eventDescription, Length: 574, dtype: int64

In [19]:
faults_diagnostics['eventDescription'].value_counts()

Low (Severity Low) Engine Coolant Level              305173
Abnormal Update Rate Tire Location                   228459
High Voltage (Fuel Level)                             77000
High Voltage (Left Fuel Level Sensor)                 76807
Low (Severity Medium) Engine Coolant Level            36109
                                                      ...  
High (Severity High) Cruise Control Enable Switch         1
Incorrect Data Aftertreatment 1 Intake %O2                1
Data May Be Invalid Front Axle Speed                      1
Incorrect Data Engine Ignition Coil #12                   1
Low Voltage (Aftertreatment 1 Purge Air Actuator)         1
Name: eventDescription, Length: 683, dtype: int64

In [28]:
active_faults_diagnostics = faults_diagnostics[faults_diagnostics['active'] == 'True']

In [38]:
faults_diagnostics['Speed'].value_counts()

0.000000     163227
0.009709       1431
0.019418        916
66.884780       836
66.836240       795
              ...  
66.375070         1
66.433320         1
63.838610         1
4.092312          1
77.817030         1
Name: Speed, Length: 13346, dtype: int64

In [39]:
active_faults_diagnostics['Speed'].value_counts()

0.000000     163227
0.009709       1431
0.019418        916
66.884780       836
66.836240       795
              ...  
66.375070         1
66.433320         1
63.838610         1
4.092312          1
77.817030         1
Name: Speed, Length: 13346, dtype: int64

In [47]:
diagnostics_drop[diagnostics_drop['spn'] == '5246']

,RecordID,ESS_Id,EventTimeStamp,eventDescription,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,LocationTimeStamp,lat_long,event_date,event_time,event_year,event_month,event_day,event_dayofweek,event_dayname,event_hour,event_time_quadrant,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,CruiseControlActive_True,IgnStatus_True,ParkingBrake_True
6133,6685,1114036,2015-02-27 16:52:12,NaN,5246,0,True,1,1486,41.033333,-77.515648,2015-02-27 16:52:49.000,"(41.033333, -77.515648)",2015-02-27,16:52:12,2015,2,27,4,Friday,16,4,6685,0.0,14.5000,64.62260,413001.400,165.2,12.0,25.52,170.43120,649.375,9368.75,74.8,63017.054230,0.766101,NaN,86.0,22527,NaN,0.000000,3276.75,NaN,0.58,0,1,0
16037,18346,1822302,2015-04-10 14:20:47,NaN,5246,0,True,1,1473,33.593934999999995,-84.385277,2015-04-10 14:21:23.000,"(33.593934999999995, -84.385277)",2015-04-10,14:20:47,2015,4,10,4,Friday,14,3,18346,0.0,14.2825,64.62260,309680.300,89.6,12.0,0.00,91.28750,0.000,6212.35,50.4,46121.798559,0.000000,NaN,87.8,5119,NaN,0.000000,3276.75,NaN,0.00,0,1,0
60580,68744,2658192,2015-05-28 11:14:03,NaN,5246,0,True,1,1795,38.983564,-84.619398,2015-05-28 11:14:38.000,"(38.983564, -84.619398)",2015-05-28,11:14:03,2015,5,28,3,Thursday,11,2,68744,0.0,14.2825,66.48672,2463.482,190.4,14.0,26.10,195.29370,599.000,50.60,82.8,306.835838,0.647223,32.0,141.8,6143,NaN,0.000000,3276.75,0.0,0.00,0,1,0
178255,198437,4535849,2015-09-10 06:45:08,NaN,5246,0,True,1,1590,37.238657,-85.92648100000001,2015-09-10 06:45:45.000,"(37.238657, -85.92648100000001)",2015-09-10,06:45:08,2015,9,10,3,Thursday,6,1,198437,0.0,14.3550,59.03026,308288.200,132.8,24.0,37.70,117.16250,657.750,7641.10,50.8,45767.411751,0.924605,NaN,98.6,22527,NaN,2.941804,NaN,NaN,1.45,0,1,0
208179,232809,5051203,2015-10-07 11:42:30,NaN,5246,0,True,1,1418,39.945324,-82.536666,2015-10-07 11:43:06.000,"(39.945324, -82.536666)",2015-10-07,11:42:30,2015,10,7,2,Wednesday,11,2,232809,67.2,14.2825,66.48672,520113.100,190.4,17.0,29.58,211.04370,997.375,10440.15,80.8,77706.869526,1.611454,NaN,127.4,22527,NaN,4.669993,NaN,NaN,1.74,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050873,1237841,119019219,2011-01-01 00:03:23,NaN,5246,0,True,1,2175,33.259027,-84.096666,2020-01-28 05:15:35.000,"(33.259027, -84.096666)",2011-01-01,00:03:23,2011,1,1,5,Saturday,0,0,1237841,0.0,14.4275,0.00000,216319.800,120.2,0.0,0.00,116.48750,0.000,4829.05,93.6,30812.896059,0.000000,NaN,93.2,6143,NaN,0.000000,NaN,100.0,0.00,0,1,1
1051657,1238712,119571469,2020-02-03 15:46:46,NaN,5246,16,True,1,2211,35.833935,-86.410925,2020-02-03 15:47:23.000,"(35.833935, -86.410925)",2020-02-03,15:46:46,2020,2,3,0,Monday,15,3,1238712,0.0,14.2825,66.48672,126087.400,179.6,11.0,17.98,223.64380,600.500,2978.40,15.2,16791.303969,0.422677,NaN,98.6,18431,NaN,0.475737,NaN,100.0,0.29,0,1,0
1052380,1239555,119959276,2020-02-06 07:45:08,NaN,5246,0,True,1,1854,35.943611,-83.823009,2020-02-06 07:45:44.000,"(35.943611, -83.823009)",2020-02-06,07:45:08,2020,2,6,3,Thursday,7,1,1239555,0.0,13.9200,66.48672,434329.400,185.0,12.0,24.36,193.38130,599.500,8664.90,66.4,59349.025288,0.581180,NaN,136.4,22527,NaN,0.000000,NaN,100.0,0.29,0,1,1
1054402,1241841,120905759,2020-02-13 13:32:39,NaN,5246,0,True,1,1872,35.707268,-81.397037,2020-02-13 13:33:15.000,"(35.707268, -81.397037)",2020-02-13,13:32:39,2020,2,13,3,Thursday,13,3,1241841,0.0,13.9925,57.78752,477486.800,197.6,10.0,21.46,204.96880,599.500,9344.30,72.4,63966.356499,0.462303,NaN,120.2,22527,NaN,0.000000,NaN,100.0,0.58,0,1,1


In [45]:
active_faults_diagnostics[active_faults_diagnostics['spn'] == '5246']

,RecordID,ESS_Id,EventTimeStamp,eventDescription,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,LocationTimeStamp,lat_long,event_date,event_time,event_year,event_month,event_day,event_dayofweek,event_dayname,event_hour,event_time_quadrant,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,CruiseControlActive_True,IgnStatus_True,ParkingBrake_True
1825,2090,1011009,2015-02-23 05:05:44,NaN,5246,0,True,1,1630,40.733009,-74.087777,2015-02-23 05:08:23.000,"(40.733009, -74.087777)",2015-02-23,05:05:44,2015,2,23,0,Monday,5,1,2090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4645.45,NaN,33470.466902,NaN,NaN,NaN,22527,NaN,NaN,NaN,NaN,NaN,0,0,0
2619,2972,1026305,2015-02-23 15:54:22,NaN,5246,0,True,1,1487,28.077361,-81.897083,2015-02-23 15:54:58.000,"(28.077361, -81.897083)",2015-02-23,15:54:22,2015,2,23,0,Monday,15,3,2972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22527,NaN,NaN,NaN,NaN,NaN,0,1,0
5182,5714,1070646,2015-02-25 13:53:08,NaN,5246,0,True,1,1329,39.399583,-82.97476800000001,2015-02-25 13:56:31.000,"(39.399583, -82.97476800000001)",2015-02-25,13:53:08,2015,2,25,2,Wednesday,13,3,5714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22527,NaN,NaN,NaN,NaN,NaN,0,1,0
5990,6535,1097942,2015-02-26 22:24:29,NaN,5246,0,True,1,1419,37.596804999999996,-85.865555,2015-02-26 22:25:05.000,"(37.596804999999996, -85.865555)",2015-02-26,22:24:29,2015,2,26,3,Thursday,22,5,6535,NaN,14.5000,64.62260,441699.6,185.0,10.0,20.30,198.16250,648.125,9087.95,57.2,69605.769379,0.634015,NaN,140.0,22527,NaN,0.000000,3276.75,NaN,0.58,0,1,0
6080,6629,1104361,2015-02-27 09:09:56,NaN,5246,0,True,1,1486,40.534259000000006,-76.43180500000001,2015-02-27 09:10:33.000,"(40.534259000000006, -76.43180500000001)",2015-02-27,09:09:56,2015,2,27,4,Friday,9,2,6629,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22527,NaN,NaN,NaN,NaN,NaN,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050873,1237841,119019219,2011-01-01 00:03:23,NaN,5246,0,True,1,2175,33.259027,-84.096666,2020-01-28 05:15:35.000,"(33.259027, -84.096666)",2011-01-01,00:03:23,2011,1,1,5,Saturday,0,0,1237841,0.0,14.4275,0.00000,216319.8,120.2,0.0,0.00,116.48750,0.000,4829.05,93.6,30812.896059,0.000000,NaN,93.2,6143,NaN,0.000000,NaN,100.0,0.00,0,1,1
1051657,1238712,119571469,2020-02-03 15:46:46,NaN,5246,16,True,1,2211,35.833935,-86.410925,2020-02-03 15:47:23.000,"(35.833935, -86.410925)",2020-02-03,15:46:46,2020,2,3,0,Monday,15,3,1238712,0.0,14.2825,66.48672,126087.4,179.6,11.0,17.98,223.64380,600.500,2978.40,15.2,16791.303969,0.422677,NaN,98.6,18431,NaN,0.475737,NaN,100.0,0.29,0,1,0
1052380,1239555,119959276,2020-02-06 07:45:08,NaN,5246,0,True,1,1854,35.943611,-83.823009,2020-02-06 07:45:44.000,"(35.943611, -83.823009)",2020-02-06,07:45:08,2020,2,6,3,Thursday,7,1,1239555,0.0,13.9200,66.48672,434329.4,185.0,12.0,24.36,193.38130,599.500,8664.90,66.4,59349.025288,0.581180,NaN,136.4,22527,NaN,0.000000,NaN,100.0,0.29,0,1,1
1054402,1241841,120905759,2020-02-13 13:32:39,NaN,5246,0,True,1,1872,35.707268,-81.397037,2020-02-13 13:33:15.000,"(35.707268, -81.397037)",2020-02-13,13:32:39,2020,2,13,3,Thursday,13,3,1241841,0.0,13.9925,57.78752,477486.8,197.6,10.0,21.46,204.96880,599.500,9344.30,72.4,63966.356499,0.462303,NaN,120.2,22527,NaN,0.000000,NaN,100.0,0.58,0,1,1


In [48]:
 spn = faults_diagnostics[faults_diagnostics['spn'] == '5246']

In [54]:
spn

,RecordID,ESS_Id,EventTimeStamp,eventDescription,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,LocationTimeStamp,lat_long,event_date,event_time,event_year,event_month,event_day,event_dayofweek,event_dayname,event_hour,event_time_quadrant,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,CruiseControlActive_True,IgnStatus_True,ParkingBrake_True
1825,2090,1011009,2015-02-23 05:05:44,NaN,5246,0,True,1,1630,40.733009,-74.087777,2015-02-23 05:08:23.000,"(40.733009, -74.087777)",2015-02-23,05:05:44,2015,2,23,0,Monday,5,1,2090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4645.45,NaN,33470.466902,NaN,NaN,NaN,22527,NaN,NaN,NaN,NaN,NaN,0,0,0
2619,2972,1026305,2015-02-23 15:54:22,NaN,5246,0,True,1,1487,28.077361,-81.897083,2015-02-23 15:54:58.000,"(28.077361, -81.897083)",2015-02-23,15:54:22,2015,2,23,0,Monday,15,3,2972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22527,NaN,NaN,NaN,NaN,NaN,0,1,0
5182,5714,1070646,2015-02-25 13:53:08,NaN,5246,0,True,1,1329,39.399583,-82.97476800000001,2015-02-25 13:56:31.000,"(39.399583, -82.97476800000001)",2015-02-25,13:53:08,2015,2,25,2,Wednesday,13,3,5714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22527,NaN,NaN,NaN,NaN,NaN,0,1,0
5277,5810,1071907,2015-02-25 14:47:00,NaN,5246,0,False,1,1329,39.399629,-82.974814,2015-02-25 14:46:56.000,"(39.399629, -82.974814)",2015-02-25,14:47:00,2015,2,25,2,Wednesday,14,3,5810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17407,NaN,NaN,NaN,NaN,NaN,0,0,0
5990,6535,1097942,2015-02-26 22:24:29,NaN,5246,0,True,1,1419,37.596804999999996,-85.865555,2015-02-26 22:25:05.000,"(37.596804999999996, -85.865555)",2015-02-26,22:24:29,2015,2,26,3,Thursday,22,5,6535,NaN,14.5000,64.62260,441699.6,185.0,10.0,20.30,198.16250,648.125,9087.95,57.2,69605.769379,0.634015,NaN,140.0,22527,NaN,0.0,3276.75,NaN,0.58,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1052382,1239564,119961467,2020-02-06 08:03:09,NaN,5246,0,False,1,1854,35.943472,-83.82324,2020-02-06 08:03:05.000,"(35.943472, -83.82324)",2020-02-06,08:03:09,2020,2,6,3,Thursday,8,2,1239564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17407,NaN,NaN,NaN,NaN,NaN,0,0,0
1054402,1241841,120905759,2020-02-13 13:32:39,NaN,5246,0,True,1,1872,35.707268,-81.397037,2020-02-13 13:33:15.000,"(35.707268, -81.397037)",2020-02-13,13:32:39,2020,2,13,3,Thursday,13,3,1241841,0.0,13.9925,57.78752,477486.8,197.6,10.0,21.46,204.96880,599.500,9344.30,72.4,63966.356499,0.462303,NaN,120.2,22527,NaN,0.0,NaN,100.0,0.58,0,1,1
1054416,1241858,120910417,2020-02-13 14:01:40,NaN,5246,0,False,1,1872,35.708101,-81.395648,2020-02-13 13:59:51.000,"(35.708101, -81.395648)",2020-02-13,14:01:40,2020,2,13,3,Thursday,14,3,1241858,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,0,0,0
1054653,1242137,121038018,2020-02-14 11:21:54,NaN,5246,19,True,88,302,38.34949,-85.708425,2020-02-14 11:22:30.000,"(38.34949, -85.708425)",2020-02-14,11:21:54,2020,2,14,4,Friday,11,2,1242137,0.0,14.7900,0.00000,334736.2,17.6,51.0,84.68,18.10625,651.500,11554.20,100.0,393.220099,2.892693,19.4,26.6,255,NaN,0.0,NaN,0.0,0.58,0,1,1


In [56]:
equipment_1329 = faults_diagnostics[faults_diagnostics['EquipmentID'] == '1329']

In [58]:
equipment_1329

,RecordID,ESS_Id,EventTimeStamp,eventDescription,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,LocationTimeStamp,lat_long,event_date,event_time,event_year,event_month,event_day,event_dayofweek,event_dayname,event_hour,event_time_quadrant,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,CruiseControlActive_True,IgnStatus_True,ParkingBrake_True
5181,5713,1070644,2015-02-25 13:53:08,NaN,4344,5,True,1,1329,39.399583,-82.97476800000001,2015-02-25 13:56:31.000,"(39.399583, -82.97476800000001)",2015-02-25,13:53:08,2015,2,25,2,Wednesday,13,3,5713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22527,NaN,NaN,NaN,NaN,NaN,0,1,0
5182,5714,1070646,2015-02-25 13:53:08,NaN,5246,0,True,1,1329,39.399583,-82.97476800000001,2015-02-25 13:56:31.000,"(39.399583, -82.97476800000001)",2015-02-25,13:53:08,2015,2,25,2,Wednesday,13,3,5714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22527,NaN,NaN,NaN,NaN,NaN,0,1,0
5183,5715,1070647,2015-02-25 13:53:08,Condition Exists Engine Protection Torque Derate,1569,31,True,1,1329,39.399583,-82.97476800000001,2015-02-25 13:56:31.000,"(39.399583, -82.97476800000001)",2015-02-25,13:53:08,2015,2,25,2,Wednesday,13,3,5715,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22527,NaN,NaN,NaN,NaN,NaN,0,1,0
5184,5716,1070648,2015-02-25 13:53:08,Low (Severity Medium) Catalyst Tank Temperature,3031,18,True,1,1329,39.399583,-82.97476800000001,2015-02-25 13:56:31.000,"(39.399583, -82.97476800000001)",2015-02-25,13:53:08,2015,2,25,2,Wednesday,13,3,5716,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22527,NaN,NaN,NaN,NaN,NaN,0,1,0
5277,5810,1071907,2015-02-25 14:47:00,NaN,5246,0,False,1,1329,39.399629,-82.974814,2015-02-25 14:46:56.000,"(39.399629, -82.974814)",2015-02-25,14:47:00,2015,2,25,2,Wednesday,14,3,5810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17407,NaN,NaN,NaN,NaN,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106958,120344,3442223,2015-07-13 13:40:49,NaN,50353,0,False,2,1329,35.057961999999996,-89.942037,2015-07-13 13:40:45.000,"(35.057961999999996, -89.942037)",2015-07-13,13:40:49,2015,7,13,0,Monday,13,3,120344,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,255,NaN,NaN,NaN,NaN,NaN,0,0,0
107433,120848,3448495,2015-07-13 19:03:15,Incorrect Data Front Operator Wiper Switch,2863,2,True,126,1329,34.467915999999995,-89.00625,2015-07-13 19:08:55.000,"(34.467915999999995, -89.00625)",2015-07-13,19:03:15,2015,7,13,0,Monday,19,4,120848,0.0,14.4275,66.48672,515409.9,192.2,20.0,14.50,215.9937,653.125,9969.25,68.0,77677.018084,1.505785,NaN,141.8,255,NaN,0.0,3276.75,NaN,1.16,0,0,1
107443,120860,3448679,2015-07-13 19:16:39,Incorrect Data Front Operator Wiper Switch,2863,2,False,126,1329,34.468055,-89.006111,2015-07-13 19:16:35.000,"(34.468055, -89.006111)",2015-07-13,19:16:39,2015,7,13,0,Monday,19,4,120860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,255,NaN,NaN,NaN,NaN,NaN,0,0,0
109243,122798,3476410,2015-07-15 06:15:16,Low (Severity Low) Engine Coolant Level,111,17,True,1,1329,35.037407,-85.58337900000001,2015-07-15 06:15:51.000,"(35.037407, -85.58337900000001)",2015-07-15,06:15:16,2015,7,15,2,Wednesday,6,1,122798,0.0,14.2100,66.48672,515946.4,95.0,36.0,38.86,89.6000,635.000,9982.15,55.6,77757.986818,1.994505,NaN,87.8,1023,NaN,0.0,3276.75,NaN,0.29,0,1,1


In [57]:
equipment_1329['eventDescription'].value_counts()

Low (Severity Low) Engine Coolant Level                                         26
Incorrect Data Front Operator Wiper Switch                                      12
Condition Exists Engine Protection Torque Derate                                 2
Low (Severity Medium) Catalyst Tank Temperature                                  2
High (Severity Low) Aftertreatment 1 Particulate Trap Intake Gas Temperature     2
High (Severity High) NOx limits exceeded due to Empty Reagent Tank               2
Low (Severity Medium) Engine Coolant Level                                       2
Low (Severity Low) Catalyst Tank Level                                           2
Name: eventDescription, dtype: int64

CHECK A COUPLE OF EQUIPMENT IDS AND SEE WHAT VALUES THEY HAD IN COMMON. MAYBE DO SOME IMPUTING SO IM NOT LOOKING AT NAN VALUES

In [50]:
spn['eventDescription'].value_counts()

Series([], Name: eventDescription, dtype: int64)

In [31]:
active_faults_diagnostics

,RecordID,ESS_Id,EventTimeStamp,eventDescription,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,LocationTimeStamp,lat_long,event_date,event_time,event_year,event_month,event_day,event_dayofweek,event_dayname,event_hour,event_time_quadrant,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,CruiseControlActive_True,IgnStatus_True,ParkingBrake_True
0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,111,17,True,2,1439,38.857638,-84.626851,2015-02-21 11:34:25.000,"(38.857638, -84.626851)",2015-02-21,10:47:13,2015,2,21,5,Saturday,10,2,1,0.0,14.2100,66.48672,423178.70000,100.4,11.0,0.00,96.74375,0.000,1632.20,43.2,12300.907429,0.000000,NaN,78.8,1023,NaN,0.000000,3276.75,NaN,0.00,0,0,1
1,2,990360,2015-02-21 11:34:34,NaN,629,12,True,127,1439,38.857638,-84.626851,2015-02-21 11:35:10.000,"(38.857638, -84.626851)",2015-02-21,11:34:34,2015,2,21,5,Saturday,11,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,0,1,0
3,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,1807,2,True,127,1369,41.421018,-87.767361,2015-02-21 11:36:08.000,"(41.421018, -87.767361)",2015-02-21,11:35:33,2015,2,21,5,Saturday,11,2,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,0,1,0
5,6,990431,2015-02-21 11:40:22,Low (Severity Low) Engine Coolant Level,111,17,True,1,1417,33.043564,-96.179722,2015-02-21 11:40:59.000,"(33.043564, -96.179722)",2015-02-21,11:40:22,2015,2,21,5,Saturday,11,2,6,48.0,14.4275,64.62260,470381.40000,181.4,30.0,38.28,196.53130,1514.500,9480.00,44.0,70349.809964,4.583399,NaN,111.2,1023,NaN,13.602200,3276.75,NaN,6.67,0,1,0
6,7,990439,2015-02-21 11:40:52,Low (Severity Low) Engine Coolant Level,111,17,True,2,1597,36.902916,-86.436481,2015-02-21 11:41:29.000,"(36.902916, -86.436481)",2015-02-21,11:40:52,2015,2,21,5,Saturday,11,2,7,82.8,14.2825,64.62260,278736.70000,188.6,80.0,39.44,210.03130,1711.375,6292.20,64.8,40961.065437,14.291750,NaN,78.8,1023,NaN,41.534780,3276.75,NaN,20.59,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1059170,1248448,123899434,2020-03-06 13:12:43,High Voltage (Fuel Level),96,3,True,126,1936,30.376851000000002,-81.74495300000001,2020-03-06 13:29:33.000,"(30.376851000000002, -81.74495300000001)",2020-03-06,13:12:43,2020,3,6,4,Friday,13,3,1248448,0.0,14.6450,66.48672,391932.60000,181.4,11.0,22.62,197.60000,597.375,8016.75,60.0,51466.131257,0.620806,NaN,120.2,1279,NaN,0.941766,NaN,100.0,1.16,0,1,0
1059173,1248452,123901805,2020-03-06 13:42:48,Low (Severity Medium) Engine Coolant Level,111,18,True,93,1886,39.015694,-77.031157,2020-03-06 13:43:24.000,"(39.015694, -77.031157)",2020-03-06,13:42:48,2020,3,6,4,Friday,13,3,1248452,0.0,14.3550,66.48672,457529.70000,181.4,11.0,19.72,207.21880,600.250,13047.05,62.0,64491.926797,0.515137,NaN,104.0,2047,NaN,5.932153,NaN,100.0,0.58,0,1,0
1059176,1248455,123905139,2020-03-06 14:04:23,Condition Exists Engine Protection Torque Derate,1569,31,True,5,1994,34.39074,-79.461805,2020-03-06 14:04:59.000,"(34.39074, -79.461805)",2020-03-06,14:04:23,2020,3,6,4,Friday,14,3,1248455,100.0,14.5000,64.62260,423937.90000,185.0,51.0,37.12,211.49370,1310.250,10722.70,96.4,58979.184416,7.647805,32.0,98.6,18431,NaN,65.010960,NaN,73.2,7.83,1,1,0
1059177,1248456,123905996,2020-03-06 14:13:38,Abnormal Rate of Change Aftertreatment 1 Intak...,3216,10,True,1,1850,34.43037,-84.920509,2020-03-06 14:14:14.000,"(34.43037, -84.920509)",2020-03-06,14:13:38,2020,3,6,4,Friday,14,3,1248456,0.0,14.3550,66.48672,465925.40000,186.8,62.0,41.18,212.84380,1340.750,9326.75,100.0,6508

In [ ]:
diagnostics_drop['active'].value_counts()

In [ ]:
#predictors = ['AcceleratorPedal', 'BarometricPressure', 'CruiseControlSetSpeed', 'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'FuelTemperature', 'IntakeManifoldTemperature', 'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure']

In [ ]:
#predictors = ['AcceleratorPedal', 'BarometricPressure', 'CruiseControlSetSpeed', 'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'FuelTemperature', 'IntakeManifoldTemperature', 'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure']
#scaler = StandardScaler().fit(diagnostics[predictors])
#diagnostics_KNN = IterativeImputer().fit(scaler.transform(diagnostics[predictors]))

In [ ]:
#diagnostics_KNN.transform(scaler.transform(diagnostics[predictors]))

In [ ]:
#diagnostics[predictors] = diagnostics_KNN.transform(scaler.transform(diagnostics[predictors]))